In [24]:
from geoband.API import *

import folium
from folium import plugins
import json
import geopandas as gpd

import pandas as pd
import numpy as np

import json

import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [ ]:
# # 데이터를 서버 주피터 노트북에 다운
# GetCompasData('SBJ_2012_001', '8', '8.오산시_유동인구(2019).csv')
# GetCompasData('SBJ_2012_001', '9', '9.오산시_어린이보호구역.csv')
# GetCompasData('SBJ_2012_001', '31', '31.오산시_법정경계(시군구).geojson')
# GetCompasData('SBJ_2012_001', '33', '33.오산시_법정경계(읍면동).geojson')

In [2]:
ppl=pd.read_csv("8.오산시_유동인구(2019).csv")
print(ppl.shape)
ppl.head()

(125163, 27)


,STD_YM,lon,lat,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,201901,126.995145,37.176703,0.08,0.05,0.04,0.04,0.08,0.27,0.80,...,0.69,0.76,0.95,0.91,0.70,0.43,0.34,0.22,0.14,0.10
1,201901,126.995142,37.177153,0.06,0.04,0.03,0.03,0.06,0.19,0.56,...,0.54,0.58,0.69,0.66,0.51,0.33,0.26,0.17,0.11,0.08
2,201901,126.995714,37.175804,0.00,0.00,0.00,0.00,0.00,0.01,0.03,...,0.04,0.04,0.05,0.05,0.03,0.02,0.02,0.01,0.01,0.00
3,201901,126.995711,37.176254,0.05,0.03,0.02,0.03,0.05,0.19,0.56,...,0.48,0.53,0.65,0.63,0.48,0.29,0.23,0.15,0.10,0.07
4,201901,126.995708,37.176705,0.02,0.01,0.01,0.01,0.02,0.07,0.18,...,0.19,0.20,0.23,0.22,0.17,0.11,0.09,0.06,0.04,0.03


In [28]:
ppl2=ppl.groupby(['lon','lat']).mean().reset_index().drop('STD_YM', axis=1)
ppl2=pd.melt(ppl2, id_vars=['lon','lat'])
ppl2=ppl2.groupby(['lon','lat']).mean().reset_index()

mms = MinMaxScaler()
ppl2['value'] = mms.fit_transform(np.array(ppl2['value']).reshape(-1,1))

ppl2

,lon,lat,value
0,126.995142,37.177153,0.004276
1,126.995145,37.176703,0.006964
2,126.995148,37.176252,0.001719
3,126.995151,37.175801,0.001137
4,126.995154,37.175351,0.001063
...,...,...,...
12397,127.097793,37.150955,0.000409
12398,127.097819,37.145997,0.003287
12399,127.098354,37.151407,0.002906
12400,127.098356,37.150956,0.025965


In [48]:
ppl2['value'].min()

0.0

In [29]:
ppl_location = []

for i in range(len(ppl2)):
    temp = []
    temp.append(ppl2.loc[i, 'lat'])
    temp.append(ppl2.loc[i, 'lon'])
    temp.append(ppl2.loc[i, 'value'])
    ppl_location.append(temp)

In [30]:
print('전체 리스트 개수:', len(ppl_location))
print('첫번째 리스트 안의 데이터 개수:', len(ppl_location[0]))

전체 리스트 개수: 12402
첫번째 리스트 안의 데이터 개수: 3


In [58]:
ssg=gpd.read_file('31.오산시_법정경계(시군구).geojson')
emd=gpd.read_file('33.오산시_법정경계(읍면동).geojson')

In [61]:
zone=pd.read_csv("9.오산시_어린이보호구역.csv")
print(zone.shape)
zone.head()

(91, 7)


,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749


In [62]:
zone_cmap={'어린이집':'orange', 
           '초등학교':'green', 
           '유치원':'blue', 
           '특수학교':'red'}
zone['marker_color']=zone['시설종류'].map(zone_cmap)

In [63]:
# 오산시 중심 맵 생성
center = [37.14887, 127.07728]
m = folium.Map(location=center, zoom_start=13)

hm = plugins.HeatMap(ppl_location, 
                     min_opacity=0.1,
                     max_val=1,
                     radius=10, blur=15,
                     max_zoom=5, color='red'
                    )

hm.add_to(m)

folium.GeoJson(emd).add_to(m)


# 어린이보호구역 시각화
for i in range(len(zone)):
    folium.Marker(
    location = zone.loc[i, ['보호구역_위도', '보호구역_경도']],
    popup = zone.loc[i, '시설명'],
    radius = 15,
    icon=folium.Icon(color=zone.loc[i, 'marker_color'], icon='child', prefix='fa')
    ).add_to(m)

m

In [64]:
m.save('어린이보호구역+평균유동인구.html')